In [1]:
import pandas as pd
import geopandas as gpd
import os
import util
idx = pd.IndexSlice

In [2]:
#import gas demand profiles 
space_heat=pd.read_csv(snakemake.input["space_profile"], header=[0,1,2], index_col=0)
water_heat=pd.read_csv(snakemake.input["water_profile"], header=[0,1,2], index_col=0)

#normalise heat profiles
space_heat=space_heat/space_heat.sum()
water_heat=water_heat/water_heat.sum()

#import household energy data
household_energy =util.load_eurostat_tsv(snakemake.input["household_energy"],
                      index_names=['nrg_bal','siec','unit','geo'])

#import gas demmand data
gas_per_zone= pd.read_csv(snakemake.input["gas_zones_housetypes"], header=0, index_col=0)

In [3]:
#combine heat and water profiles
gas_profile=pd.concat([water_heat, space_heat], axis=1).sort_index(axis=1)

In [4]:
#select consumption of natural gas in TJ in UK
household_energy.reset_index()
household_energy=household_energy.query('siec == "G3000" & geo == "UK" & unit == "TJ"').T
household_energy.columns =['total', 'cooking', 'space_heating', 'water_heating']

#calculate ratios
household_energy['ratio_space_heating']=household_energy['space_heating']/household_energy['total']
household_energy['ratio_water_heating']=household_energy['water_heating']/household_energy['total']


In [5]:
# create dataframe with anual gas consumption
anual_gas= gas_per_zone.loc[:, ['non-domestic_weighted','gas_SFH','gas_MFH']]
anual_gas.columns=['gas_COM','gas_SFH', 'gas_MFH']

In [6]:
#calculate gas demand by house type and end use
anual_gas['SFH_space_heating']= anual_gas['gas_SFH']*household_energy.loc['2014 ', 'ratio_space_heating']
anual_gas['SFH_water_heating']= anual_gas['gas_SFH']*household_energy.loc['2014 ', 'ratio_water_heating']

anual_gas['MFH_space_heating']= anual_gas['gas_MFH']*household_energy.loc['2014 ', 'ratio_space_heating']
anual_gas['MFH_water_heating']= anual_gas['gas_MFH']*household_energy.loc['2014 ', 'ratio_water_heating']

anual_gas['COM_space_heating']= anual_gas['gas_COM']*household_energy.loc['2014 ', 'ratio_space_heating']
anual_gas['COM_water_heat ing']= anual_gas['gas_COM']*household_energy.loc['2014 ', 'ratio_water_heating']

anual_gas.drop(['gas_COM','gas_SFH', 'gas_MFH'], axis=1, inplace=True)

In [7]:
#transform dataframe to multi-level column index dataframe
anual_gas=anual_gas.T

#add building types and end uses
building_type=pd.Series(['SFH', 'SFH', 'MFH', 'MFH', 'COM', 'COM'], index=anual_gas.index)
end_uses=pd.Series(['space_heat', 'water_heat', 'space_heat', 'water_heat', 'space_heat', 'water_heat'],
                   index=anual_gas.index)
anual_gas=anual_gas.join([building_type,end_uses])

#set indices
anual_gas.reset_index(drop=False)
anual_gas.set_index([0, 1], inplace=True)
anual_gas.index.names=['building', 'end-use']

#stack to get multilevel Series
anual_gas=anual_gas.stack()

In [8]:
#make sure indices are identical
anual_gas.index=anual_gas.index.swaplevel(2,0)
anual_gas.index=anual_gas.index.swaplevel(2,1)

gas_profile.columns.names=['country', 'building', 'end-use']
anual_gas.index.names=['country', 'building', 'end-use']

#multiply anual gas demand and export to .csv
gas_profile=gas_profile.multiply(anual_gas)

# calculate heat demand from gas demand
heat_demand= (gas_profile*-0.97)
#agregate based on zones
heat_demand=heat_demand.sum(level='country', axis=1)

#export to csv
heat_demand.index=pd.to_datetime(heat_demand.index)
heat_demand.to_csv(snakemake.output[0])